In [12]:
import hashlib

content = """In computer science, consistent hashing is a special kind of 
hashing such that when a hash table is resized, only K/n keys need to be 
remapped on average, where K is the number of keys, and n is the number of 
slots. In contrast, in most traditional hash tables, a change in the number 
of array slots causes nearly all keys to be remapped because the mapping 
between the keys and the slots is defined by a modular operation."""

servers = [
    "10.10.1.1",
    "10.10.2.2",
    "10.10.3.3",
    "10.10.4.4",
]


class HashRing:
    def __init__(self, nodes=None, replicas=3):
        """
        initialize
        """
        self.replicas = replicas
        self.ring = dict()
        self._sorted_keys = []

        if nodes:
            for node in nodes:
                self.add_node(node)
                
    def add_node(self, node):
        """
        Adds a `node` to the hash ring (including a number of replicas)
        """
        for i in range(self.replicas):
            virtual_node = f"{node}#{i}"
            key = self.gen_key(virtual_node)
            self.ring[key] = node
            self._sorted_keys.append(key)
            # print(f"{virtual_node} --> {key} --> {node}")

        self._sorted_keys.sort()
        # print([self.ring[key] for key in self._sorted_keys])

    def remove_node(self, node):
        """
        Removes `node` from the hash ring and its replicas
        """
        for i in range(self.replicas):
            key = self.gen_key(f"{node}#{i}")
            del self.ring[key]
            self._sorted_keys.remove(key)

    def get_node(self, string_key):
        """
        Given a string key， a corresponding node in the hash ring is returned.

        If the hash ring is empty, `None` is returned.
        """
        return self.get_node_pos(string_key)[0]

    def get_node_pos(self, string_key):
        """
        Given a string key， a corresponding node in the hash ring is returned
        along with it's position in the ring.

        If the hash ring is empty, (`None`, `None`) is returned.
        """
        if not self.ring:
            return None, None

        key = self.gen_key(string_key)
        nodes = self._sorted_keys
        for i in range(len(nodes)):
            node = nodes[i]
            if key < node:
                return self.ring[node], i

       
        return self.ring[nodes[0]], 0

    def gen_key(self, string_key):
        """
        Given a string key, it returns a long value, this long value represents
        a place on the hash ring
        """
        m = hashlib.md5()
        m.update(string_key.encode('utf-8'))  
        return m.hexdigest() 

    
    def redistribute_objects(self, node):
       # TODO：think about how to redistribute objects after add or delete the node
         pass

def consistent_hash(replicas):
    hr = HashRing(servers, replicas)
    words = content.split()

    database = {s: [] for s in servers}

    for w in words:
        database[hr.get_node(w)].append(w)

    # print(f"words={len(words)}n")

    for node, result in database.items():
        print(f"{node}={len(result)}nresult={result}")


if __name__ == '__main__':
    consistent_hash(3)

10.10.1.1=39nresult=['hashing', 'is', 'of', 'hashing', 'such', 'table', 'is', 'only', 'K/n', 'be', 'where', 'K', 'is', 'the', 'number', 'of', 'keys,', 'and', 'n', 'is', 'the', 'number', 'of', 'slots.', 'traditional', 'the', 'number', 'of', 'slots', 'nearly', 'all', 'be', 'the', 'the', 'and', 'the', 'slots', 'is', 'defined']
10.10.2.2=3nresult=['that', 'average,', 'between']
10.10.3.3=33nresult=['In', 'computer', 'science,', 'a', 'special', 'kind', 'when', 'a', 'hash', 'resized,', 'keys', 'need', 'to', 'remapped', 'on', 'In', 'in', 'most', 'hash', 'a', 'change', 'in', 'array', 'causes', 'keys', 'to', 'remapped', 'mapping', 'keys', 'by', 'a', 'modular', 'operation.']
10.10.4.4=4nresult=['consistent', 'contrast,', 'tables,', 'because']


In [ ]:
import zmq
import sys
# server
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5556")

while True:
    message = socket.recv()
    print("Received: %s" % message)
    socket.send(b"I am server, i have receive your message")
    
# to do
# if the sever goes down, we need send the request to another node, but if the client goes down, do we need take some action?

# server works on Request-Reply mode

In [ ]:
# sever 
import gevent
import time
import zmq.green as zmq

_BINDING = 'tcp://127.0.0.1:7000'
context = zmq.Context()

def server():
    server_socket = context.socket(zmq.REP)
    server_socket.bind(_BINDING)

    while True:
        received = server_socket.recv()
        print("Received: [{}]\n".format(received))
        server_socket.send_string('TestResponse')

server = gevent.spawn(server)
server.join()

# server works on Publish-Subscribe mode

In [12]:
import zmq
import time
 
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://*:7000")
 
for i in range(10):
    print('send message...' + str(i))
    message = 'message' + str(i) 
    socket.send(bytes(message, encoding = "utf8"))
    time.sleep(1)

send message...0
send message...1
send message...2
send message...3
send message...4
send message...5
send message...6
send message...7
send message...8
send message...9
